In [ ]:
from sklearn.datasets import load_boston

import matplotlib.pyplot as plt

In [ ]:
boston = load_boston()
X = boston.data
y = boston.target
print(boston.DESCR)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf = RandomForestRegressor(n_estimators=100, 
                           max_features=10, 
                           min_samples_split=2,
                           random_state=10)
rf.fit(X,y)
score = cross_val_score(rf,X,y,cv = 5)
print('score: mean={:.2e} ; std={:.2e}'\
      .format(score.mean(),score.std()))
    

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'n_estimators' : [100,1000],
    'max_features' : [10,13],
    'min_samples_split' : [2,10]
}
clf = GridSearchCV(rf, parameters, cv=5, verbose=1,n_jobs=4)

clf.fit(X,y)
print('GridSearch finished')

In [ ]:
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))


In [ ]:
plt.bar(range(len(means)),means)
plt.ylim(.6,.7)
plt.show()